In [1]:
from typing import Optional, List


class State:
    """
    Represents a state in the Farmer's Game where certain items are either on the left or right side of the river.

    Each state keeps track of the items' positions (whether they are on the left side or not) and a reference
    to the previous state, allowing for backtracking during the search for a solution.

    Attributes:
        itemNames (tuple[str, ...]): A class-level attribute that stores the names of the items involved in the game.
            It is initialized with a default value of "Itemnames not set" and can be updated using the `addItemNames` method.
        itemsLeft (List[bool]): A list of boolean values indicating the position of each item.
            `True` means the item is on the left side, and `False` means it is on the right.
        prev (Optional[State]): A reference to the previous state, allowing the construction of a path through the states.
    """

    itemNames: tuple[str, ...] = tuple("Itemnames not set")

    def __init__(self, itemsLeft: List[bool], prev: Optional["State"] = None):
        """
        Creates a `State` object representing the positions of items based on a list of booleans.

        Each boolean value indicates whether a specific item is on the left side (`True`) or not (`False`).
        Optionally, a reference to the previous `State` can be provided to enable backtracking.

        Args:
            itemsLeft (List[bool]): A list of booleans where `True` represents the item being on the left side,
                and `False` represents the item being on the right side.
            prev (Optional["State"], optional): A reference to the previous `State` object, useful for backtracking through states.
                Defaults to None.
        """
        self.itemsLeft = itemsLeft
        self.prev = prev

    @classmethod
    def addItemNames(cls, itemNames: tuple[str, ...]) -> None:
        """
        Sets the names of the items for the `State` class.

        This class method allows the assignment of item names to the class-level attribute `itemNames`.
        These names are used to describe each item in the state representation of the game.

        Args:
            itemNames (tuple[str, ...]): A tuple containing the names of the items. The length of this tuple should match the
                number of items in the game (or the number of boolean values in the `itemsLeft` attribute of each `State`).

        """
        cls.itemNames = itemNames

    def __str__(self) -> str:
        return (
            "["
            + ", ".join(
                [
                    self.itemNames[i]
                    for i in range(len(self.itemNames))
                    if self.itemsLeft[i]
                ]
            )
            + "]"
        )

    def __eq__(self, other) -> bool:
        if isinstance(other, State):
            return self.itemsLeft == other.itemsLeft
        return False

    def __hash__(self):
        # This allows the state to be used in a set or as a dictionary key
        return hash(tuple(self.itemsLeft))

    def getNeighbours(self) -> List["State"]:
        """
        Finds all neighboring states based on the current state's configuration.

        A neighboring state is one where:
        - Either only the first item (farmer) moves from one side to the other, or
        - The first item (farmer) moves together with an item from the same side.

        Returns:
            List[State]: A list of neighboring states derived from the current state.
        """
        neighbours: List[State] = []
        prev: State = self

        # farmer is on the right
        if not self.itemsLeft[0]:
            self.itemsLeft[0] = True
            neighbour = State(self.itemsLeft.copy(), prev)
            neighbours.append(neighbour)

            for i in range(1, len(self.itemsLeft)):
                if not self.itemsLeft[i]:
                    self.itemsLeft[i] = True
                    neighbour = State(self.itemsLeft.copy(), prev)
                    neighbours.append(neighbour)
                    self.itemsLeft[i] = False
            self.itemsLeft[0] = False

        # farmer is on the left
        if self.itemsLeft[0]:
            self.itemsLeft[0] = False
            neighbour = State(self.itemsLeft.copy(), prev)
            neighbours.append(neighbour)

            for i in range(1, len(self.itemsLeft)):
                if self.itemsLeft[i]:
                    self.itemsLeft[i] = False
                    neighbour = State(self.itemsLeft.copy(), prev)
                    neighbours.append(neighbour)
                    self.itemsLeft[i] = True
            self.itemsLeft[0] = True

        return neighbours


In [2]:
from typing import List, Set, Optional, Deque
from collections import deque
from collections.abc import Iterable


class FarmerGame:
    """
    A class to represent a farmer's game.

    Attributes:
        itemNames (tuple[str, ...]): The names of the items in the game.
        badStates (set[State]): A set of states that are invalid.
        source (State): The starting state of the game.
        target (State): The target state of the game.
    """

    def __init__(
        self, itemNames: tuple[str, ...], badStates: Optional[Set[State]] = None
    ) -> None:
        """
        Initializes a Farmer's Game with the provided items and optionally defines bad states.

        This method sets up the game based on a tuple of `itemNames`, which represent the items involved in the game.
        The first item in the tuple is always the item (Farmer), that is required to cross the river with any other items.
        Additionally, a set of "bad" or forbidden states can be provided to specify configurations that are not allowed.

        Args:
            itemNames (tuple[str, ...]): A tuple of strings representing the names of the items in the game.
                The first item in the tuple is the farmer, who must always accompany other items during crossings.
            badStates (Optional[Set[State]], optional): A set of states that are considered illegal and should be avoided.
                Defaults to None if no bad states are specified.
        """

        self.itemNames = itemNames
        if not badStates:
            self.badStates: Set[State] = set()
        else:
            self.badStates = badStates
        self.source: Optional[State] = None
        self.target: Optional[State] = None

    def setSource(self, source: State) -> None:
        if source in self.badStates:
            raise ValueError("Source State is a bad State")
        else:
            self.source = source

    def setTarget(self, target: State) -> None:
        if target in self.badStates:
            raise ValueError("target State is a bad State")
        else:
            self.target = target

    def addBadStates(self, badStates: Iterable[State]) -> None:
        """
        Adds all states in the given Iterable to `badStates`.

        Args:
            badStates (Iterable[State]): Iterable of states to be added to the `badStates` attribute of `FarmerGame`.

        Raises:
            ValueError: If a state equal to the source state is attempted to be added.
            ValueError: If a state equal to the target state is attempted to be added.
        """
        for state in badStates:
            if state == self.source:
                raise ValueError(
                    f"Attempted to add state {state} which is the source state"
                )
            elif state == self.target:
                raise ValueError(
                    f"Attempted to add state {state} which is the target state"
                )
            self.badStates.add(state)

    def __backTrack(self, endState: State) -> tuple[List[State], bool]:
        """
        Private method to compute a path to the specified end state by tracing back through its predecessors.

        This method utilizes the `prev` attribute of `State` objects to reconstruct the sequence of states
        leading to the given `endState`, effectively creating a path from the starting state to the target.

        Args:
            endState (State): The state for which the path is to be computed, based on its successive predecessors.

        Returns:
            tuple(List[State], bool): A tuple where the first element is a list of `State` objects representing the
            path from the source to the target (if found), and the second element is a boolean indicating whether
            the path was successfully found. If no path exists, the return value defaults to ([], False).
        """

        path: List[State] = [endState]
        while endState.prev:
            path.append(endState.prev)
            endState = endState.prev
        return path[::-1], True

    def __movesFromPath(self, path: List[State]) -> None:
        """
        private method that returns the actions required to take for the given path using the following format:
        f"Move {rightMovedItems} right" or f"Move {leftMovedItems} left".

        Args:
            path (List[State]): list of states representing a path.

        Returns:
            None: this method does not return any value.
        """
        print(f"Actions required to reach State {self.target} from {self.source}:")

        for i in range(len(path) - 1):
            curr: State = path[i]
            next: State = path[i + 1]
            leftIdx: List[int] = []
            rightIdx: List[int] = []

            for idx, values in enumerate(zip(curr.itemsLeft, next.itemsLeft)):
                diff: int = int(values[1]) - int(values[0])
                if diff == 0:
                    continue
                elif diff == 1:
                    leftIdx.append(idx)
                elif diff == -1:
                    rightIdx.append(idx)

            leftMovedItems: str = ", ".join([curr.itemNames[idx] for idx in leftIdx])
            rightMovedItems: str = ", ".join([curr.itemNames[idx] for idx in rightIdx])
            if leftMovedItems:
                print(f"Move {leftMovedItems} left")
            elif rightMovedItems:
                print(f"move {rightMovedItems} right")

    def bfs(self, printActions: bool = False) -> tuple[List[State], bool]:
        """
        Performs a breadth-first search (BFS) to find a valid path from the source state to the target state.

        Args:
            printActions (bool, optional): If True, prints the sequence of actions required to go from the source state
                to the target state. Defaults to False.

        Raises:
            ValueError: If the source state is not specified.
            ValueError: If the target state is not specified.

        Returns:
            tuple(List[State], bool): A tuple where the first element is the list of states representing the path from
            the source to the target (if found), and the second element is a boolean indicating whether the search
            was successful. If no path is found, defaults to ([], False).

        """

        if not isinstance(self.source, State):
            raise ValueError("Source is not specified")
        if not isinstance(self.target, State):
            raise ValueError("Target is not specified")
        q: Deque[State] = deque()
        visited: Set[State] = set([self.source])
        q.append(self.source)

        while q:
            curr: State = q.popleft()
            if curr == self.target:
                path: List[State]
                succes: bool
                path, succes = self.__backTrack(curr)
                if printActions:
                    self.__movesFromPath(path)
                    return path, succes
                else:
                    return path, succes

            for neighbour in curr.getNeighbours():
                if neighbour not in visited:
                    if neighbour not in self.badStates:
                        visited.add(neighbour)
                        q.append(neighbour)

        return [], False


In [3]:
from typing import Set, List


def gameReader(filePath: str) -> FarmerGame:
    """
    Creates a `FarmerGame` object from a data file.

    The data file should contain the following space-separated information:
    1. A list of item names.
    2. A binary representation of the source state, where `1` at position `i` indicates that item `i` is on the left side.
    3. A binary representation of the target state, where `1` at position `i` indicates that item `i` is on the left side.

    This method reads the item names, the source state, and the target state from the file. It does not initialize the `badStates`
    property; that is done separately via the `badStateReader`.

    ### Example of a valid data file:
        Farmer Wolf Goat Cabbage
        0 0 0 0
        1 1 1 1

    Args:
        filePath (str): Path to the file containing the data for the `FarmerGame` object.

    Returns:
        FarmerGame: The `FarmerGame` object accosiated with the given input file
    """
    boolMapping = lambda x: x == "1"

    with open(filePath, "r") as file:
        lines: List[str] = file.readlines()
        itemNames: tuple[str, ...] = tuple(lines[0].split())
        game = FarmerGame(itemNames)
        State.addItemNames(itemNames)

        source: State = State(list(map(boolMapping, lines[1].split())))
        target: State = State(list(map(boolMapping, lines[2].split())))

        game.setSource(source)
        game.setTarget(target)

    return game


def badStateReader(filePath: str) -> Set[State]:
    """
    Creates a set of states from a file

    the file should contain the following space seperated info:
    1. A number `n` saying how many badSates will be listed below
    2. The next `n` lines represent the binary configurations of the bad states, where `1` at position `i`
    indicates that item `i` is on the left side, and `0` indicates that the item is on the right side.
    ### Example for the Farmer Wolf Goat Cabbage problem :
        6
        0 1 1 0
        0 1 1 1
        0 0 1 1
        1 0 0 1
        1 0 0 0
        1 1 0 0


    Args:
        filePath (str): path to the file containing the data for the badStates.

    Returns:
        Set[State]: A set of `State` objects representing the bad states.
    """
    boolMapping = lambda x: x == "1"
    with open(filePath, "r") as file:
        lines = file.readlines()
        nBadStates: int = int(lines[0])
        badStates: set[State] = set()
        for i in range(nBadStates):
            state = State(list(map(boolMapping, lines[i + 1].split())))
            badStates.add(state)
    return badStates


In [4]:
from typing import Set


if __name__ == "__main__":

    game: FarmerGame = gameReader(
        "./data/Data.txt"
    )  # change to ./data/alphabetData.txt for example 2 or ./data/piratesData.txt for example 3
    badStates: Set[State] = badStateReader(
        "./data/BadStates.txt"
    )  # change to ./data/alphabetBadStates.txt for example 2 or ./data/piratesBadStates.txt for example 3
    game.addBadStates(badStates)

    print(f"starting state of the game: {game.source}")
    print(f"ending state of the game: {game.target}")

    path, result = game.bfs(printActions=True)
    if result:
        print(
            f"path from start to end found with {len(path) - 1} steps: {list(map(str, path))}"
        )
    else:
        print(f"No path from {game.source} to {game.target} found")


starting state of the game: []
ending state of the game: [Farmer, Wolf, Goat, Cabbage]
Actions required to reach State [Farmer, Wolf, Goat, Cabbage] from []:
Move Farmer, Goat left
move Farmer right
Move Farmer, Wolf left
move Farmer, Goat right
Move Farmer, Cabbage left
move Farmer right
Move Farmer, Goat left
path from start to end found with 7 steps: ['[]', '[Farmer, Goat]', '[Goat]', '[Farmer, Wolf, Goat]', '[Wolf]', '[Farmer, Wolf, Cabbage]', '[Wolf, Cabbage]', '[Farmer, Wolf, Goat, Cabbage]']
